# 使用 Activeloop 的 DeepLake 进行问答
在本教程中，我们将使用 Langchain + Activeloop 的 Deep Lake 与 GPT4 一起对群聊进行语义搜索和提问。

在[这里](https://twitter.com/thisissukh_/status/1647223328363679745)查看演示。

## 1. 安装所需的包

In [ ]:
!python3 -m pip install --upgrade langchain 'deeplake[enterprise]' openai tiktoken

## 2. 添加 API 密钥

In [2]:
import getpass
import os

from langchain.chains import RetrievalQA
from langchain_community.vectorstores import DeepLake
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
activeloop_token = getpass.getpass("Activeloop Token:")
os.environ["ACTIVELOOP_TOKEN"] = activeloop_token
os.environ["ACTIVELOOP_ORG"] = getpass.getpass("Activeloop Org:")

org_id = os.environ["ACTIVELOOP_ORG"]
embeddings = OpenAIEmbeddings()

dataset_path = "hub://" + org_id + "/data"



## 2. 创建示例数据

你可以使用以下提示通过 ChatGPT 生成一个示例群聊对话：

```
生成一个三个朋友谈论他们一天的群聊对话，引用真实地点和虚构名字。让它有趣且尽可能详细。
```

我已经在 `messages.txt` 中生成了这样的聊天记录。我们可以保持简单，在我们的示例中使用这个。

## 3. 摄入聊天嵌入

我们加载文本文件中的消息，分块并上传到 ActiveLoop 向量存储。

In [4]:
with open("messages.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
pages = text_splitter.split_text(state_of_the_union)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.create_documents(pages)

print(texts)

dataset_path = "hub://" + org_id + "/data"
embeddings = OpenAIEmbeddings()
db = DeepLake.from_documents(
    texts, embeddings, dataset_path=dataset_path, overwrite=True
)

[Document(page_content='Participants:\n\nJerry: Loves movies and is a bit of a klutz.\nSamantha: Enthusiastic about food and always trying new restaurants.\nBarry: A nature lover, but always manages to get lost.\nJerry: Hey, guys! You won\'t believe what happened to me at the Times Square AMC theater. I tripped over my own feet and spilled popcorn everywhere! 🍿💥\n\nSamantha: LOL, that\'s so you, Jerry! Was the floor buttery enough for you to ice skate on after that? 😂\n\nBarry: Sounds like a regular Tuesday for you, Jerry. Meanwhile, I tried to find that new hiking trail in Central Park. You know, the one that\'s supposed to be impossible to get lost on? Well, guess what...\n\nJerry: You found a hidden treasure?\n\nBarry: No, I got lost. AGAIN. 🧭🙄\n\nSamantha: Barry, you\'d get lost in your own backyard! But speaking of treasures, I found this new sushi place in Little Tokyo. "Samantha\'s Sushi Symphony" it\'s called. Coincidence? I think not!\n\nJerry: Maybe they named it after your a

Your Deep Lake dataset has been successfully created!


\

Dataset(path='hub://adilkhan/data', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (3, 1536)  float32   None   
    id        text      (3, 1)      str     None   
 metadata     json      (3, 1)      str     None   
   text       text      (3, 1)      str     None   


`可选`：你也可以使用 Deep Lake 的托管张量数据库作为托管服务并在那里运行查询。为此，需要在创建向量存储时指定运行时参数为 {'tensor_db': True}。这个配置允许在托管张量数据库上执行查询，而不是在客户端。应该注意的是，这个功能不适用于本地存储或内存中的数据集。如果向量存储已经在托管张量数据库之外创建，可以按照规定的步骤将其转移到托管张量数据库。

In [5]:
# with open("messages.txt") as f:
#     state_of_the_union = f.read()
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# pages = text_splitter.split_text(state_of_the_union)

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
# texts = text_splitter.create_documents(pages)

# print(texts)

# dataset_path = "hub://" + org + "/data"
# embeddings = OpenAIEmbeddings()
# db = DeepLake.from_documents(
#     texts, embeddings, dataset_path=dataset_path, overwrite=True, runtime={"tensor_db": True}
# )

## 4. 提问

现在我们可以通过语义搜索提出问题并得到答案：

In [ ]:
db = DeepLake(dataset_path=dataset_path, read_only=True, embedding=embeddings)

retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["k"] = 4

qa = RetrievalQA.from_chain_type(
    llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=False
)

# What was the restaurant the group was talking about called?
query = input("Enter query:")

# The Hungry Lobster
ans = qa({"query": query})

print(ans)